In [7]:
! pip install kaggle datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 14.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is

In [8]:
import os
import shutil

# Create the .kaggle directory
os.makedirs('/root/.kaggle', exist_ok=True)

# Now copy the kaggle.json file to the .kaggle folder
shutil.copy('/content/kaggle.json', '/root/.kaggle/kaggle.json')

'/root/.kaggle/kaggle.json'

In [9]:
!kaggle datasets download -d mohammadnouralawad/spider-text-sql

Dataset URL: https://www.kaggle.com/datasets/mohammadnouralawad/spider-text-sql
License(s): CC-BY-SA-4.0
spider-text-sql.zip: Skipping, found more recently modified local copy (use --force to force download)


In [10]:
import zipfile
# Replace 'your_zip_file.zip' with the actual path to your zip file
with zipfile.ZipFile('/content/spider-text-sql.zip', 'r') as zip_ref:
    zip_ref.extractall('extracted_files') # Replace 'extracted_files' with desired extraction location

In [14]:
import pandas as pd
df = pd.read_csv('/content/extracted_files/spider_text_sql.csv')
df.head()

,text_query,sql_command
0,How many heads of the departments are older th...,SELECT count(*) FROM head WHERE age > 56
1,"List the name, born state and age of the heads...","SELECT name , born_state , age FROM head ORD..."
2,"List the creation year, name and budget of eac...","SELECT creation , name , budget_in_billions ..."
3,What are the maximum and minimum budget of the...,"SELECT max(budget_in_billions) , min(budget_i..."
4,What is the average number of employees of the...,SELECT avg(num_employees) FROM department WHER...


In [15]:
import pandas as pd
from datasets import Dataset, DatasetDict

# Sample DataFrame (Replace this with your actual data)
df = df

# Concatenate columns into a single "text" column
df["text"] = "### Text Query: " + df["text_query"] + "\n### SQL Command: " + df["sql_command"]

# Convert to Hugging Face Dataset
hf_dataset = Dataset.from_pandas(df)

# Remove the auto-generated index column
hf_dataset = hf_dataset.remove_columns(["__index_level_0__"]) if "__index_level_0__" in hf_dataset.column_names else hf_dataset

# **Split dataset into Train (80%) and Validation (20%)**
split_dataset = hf_dataset.train_test_split(test_size=0.2, seed=42)

# Convert to DatasetDict (For Trainer Compatibility)
dataset_dict = DatasetDict({
    "train": split_dataset["train"],
    "validation": split_dataset["test"]
})

# Print dataset sizes
print(f"Train size: {len(dataset_dict['train'])}, Validation size: {len(dataset_dict['validation'])}")

# Print sample from train set
print(dataset_dict["train"][0])

Train size: 6427, Validation size: 1607
{'text_query': 'What are the name and location of the cinema with the largest capacity?', 'sql_command': 'SELECT name ,  LOCATION FROM cinema ORDER BY capacity DESC LIMIT 1', 'text': '### Text Query: What are the name and location of the cinema with the largest capacity?\n### SQL Command: SELECT name ,  LOCATION FROM cinema ORDER BY capacity DESC LIMIT 1'}


In [16]:
import json

# Number of rows to store as JSON
num_rows_to_store = 5

# Create a list to store the JSON objects
json_data = []

# Iterate through the first 'num_rows_to_store' rows of the train dataset
for i in range(num_rows_to_store):
    # Convert each row to a dictionary
    row_dict = dataset_dict["train"][i]

    # Append the dictionary to the JSON data list
    json_data.append(row_dict)

# Save the JSON data to a file
with open("train_data.json", "w") as f:
    json.dump(json_data, f, indent=4)  # Use indent for pretty printing